In [1]:
%load_ext autoreload
%autoreload 2

import os,sys,warnings
import h5py, pandas
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
np.set_printoptions(legacy='1.25')
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 24
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 6
mpl.rcParams['errorbar.capsize'] = 6
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# mpl.rcParams.update({"axes.grid" : True})
import util as yu
yu.flag_fast=False

enss=['b']
ens2full={'a24':'cA211.53.24','a':'cA2.09.48','b':'B64','c':'C80','d':'D96'}
ens2label={'a24':'A24','a':'A48','b':'B64','c':'C80','d':'D96'}
ens2a={'a24':0.0908,'a':0.0938,'b':0.07957,'c':0.06821,'d':0.05692} # fm
ens2N={'a24':24,'a':48,'b':64,'c':80,'d':96}
ens2N_T={'a24':24*2,'a':48*2,'b':64*2,'c':80*2,'d':96*2}

# ens2mN={'a':931}
# ens2mpiC={'a':131}
# ens2mpi0={'a':111}

ens2amu={'a24':0.0053,'a':0.0009,'b':0.00072}
ens2ZP={'a24':(0.4670,0.0046),'a':(0.500,0.030)}

hbarc = 1/197.3
ens2aInv={ens:1/(ens2a[ens]*hbarc) for ens in enss} # MeV

ens='b'

figPath='fig/'

In [3]:
shift131=(2.9,0.5); shiftMpi0=(0,0); shiftMpiC=(-3.1,0.5)
doShift=False

#[label,sgm,flavor,a-correction,shift]
sgms={
    # lattice matrix element
    'this':['This work',(36.3,0.7),'2','a2',shift131], # mpi=131 O(a)-improved
    
    'this2':['This work',(34.4,1.3),'2','a2',shift131],
    'ETM25':['ETM25',(41.9,8.1),'211','a3',shiftMpiC], # Alexandrou:2024ozj mpi=140,137,141 O(a)-improved + a^2-extrapolation
    'ETM19':['ETM19',(41.6,3.8),'211','a2',shiftMpiC], # Alexandrou:2019brg mpi=140
    # 'ETM19_A2':['ETM19',()], # A2
    'Mainz23':['Mainz23',(43.7,3.6),'21','a2',shiftMpi0], # Agadjanov:2023efe mpi@134.8
    'Mainz23_wd':['Mainz23',(42.3,2.4),'21','a2',shiftMpi0], # Agadjanov:2023efe
    'Mainz23_prior':['Mainz23',(46.9,1.7),'21','a2',shiftMpi0], # Agadjanov:2023efe
    
    'PNDME21':['PNDME21',(41.9,4.9),'211','a2',shiftMpi0], # Gupta:2021ahb standard mpi=135
    'PNDME21_prior':['PNDME21',(59.6,7.4),'211','a2',shiftMpi0], # with prior from ChiPT
    
    'PNDME25':['PNDME25',(42,6),'211','a2',shiftMpi0], # Park:2025rxi standard mpi=135
    'PNDME25_prior':['PNDME25',(62,6),'211','a2',shiftMpi0], # with prior from ChiPT
    
    # lattice FH
    'BMW20':['BMW20',(37.4,3.0,4.1),'1111','a?',shiftMpi0], # Borsanyi:2020bpd mpi=134.8
    'RQCD23':['RQCD23',(43.9,4.7),'21','a3',shiftMpi0], # Bali:2023sdi mpi=134.8
    'QCDSF12':['QCDSF12',(37,8,6),'2','a2?',shiftMpiC], # Bali:2012qs mpi=138
    
    'Liang25':['Liang25',(48.1,1.9),'21','a2',None], # Liang:2025adz
    'Liang25_twoLoop':['Liang25',(55.9,2.5),'21','a2',None],
    
    # phenomenology
    'Hofe23':['Hoferichter 23',(59.0,3.5),'phe','0',shiftMpiC], # Hoferichter:2023ptl; scattering length from pionic atom
    'Ruiz18':['Ruiz de Elvira 18',(58,5),'phe','0',shiftMpiC], # RuizdeElvira:2017stg; low-energy piN cross section
    'Frie19':['Friedman 19',(57,7),'phe','0',shiftMpiC], # Friedman:2019zhc; pionic atom with large Z used
}

fla2sty={'2':('r','s'),'21':('g','o'),'211':('b','d'),'1111':('b','d'),'phe':('purple','x')}

fig, axs = yu.getFigAxs(1,1,sharey=True)
ax=axs[0,0]

cases=['PNDME25','ETM25','Mainz23_wd','PNDME21','Liang25','RQCD23','BMW20','QCDSF12','Hofe23','Frie19','Ruiz18']

xlim=[10,95]
ax.set_xlim(xlim)
ax.set_xticks([20,40,60,80])
ax.set_xlabel(r'$\sigma_{\pi N}$ [MeV]',fontsize=20)
ylim=[-len(cases),1]
ax.set_ylim(ylim)


for i,case in enumerate(cases):
    if case is None:
        continue
    label,me,fla,_,shift=sgms[case][:5]
    sty=fla2sty[fla]
    sty=('r','s') if i<4 else ('g','o') if i<8 else ('b','d')
    
    mfc='white' if case in ['Mainz23_prior','PNDME25_prior'] else None
    
    if doShift:
        if len(me)==2:
            me=(me[0]+shift[0],me[1],shift[1])
        elif len(me)==3:
            me=(me[0]+shift[0],me[1],np.sqrt(me[2]**2+shift[1]**2))
    
    if len(me)==2:
        ax.errorbar(x=me[0] ,xerr=me[1],y=[-i], color=sty[0],fmt=sty[1], mfc=mfc)
    elif len(me)==3:
        errTot=np.sqrt(me[1]**2+me[2]**2)
        if doShift:
            print(case,me,errTot)
        ax.errorbar(x=me[0] ,xerr=me[1],y=[-i], color=sty[0],fmt=sty[1], mfc=mfc)
        ax.errorbar(x=me[0] ,xerr=errTot,y=[-i], color=sty[0],fmt=sty[1], mfc=mfc)
    else:
        1/0
    if case=='this':
        mean=me[0]; err=me[1] if len(me)==2 else np.sqrt(me[1]**2+me[2]**2)
        ax.fill_betweenx(y=ylim, x1=mean-err, x2=mean+err, alpha=0.2, color='red')

ys=[0,2,3,4]
cases2=['PNDME25_prior','Mainz23_prior','PNDME21_prior','Liang25_twoLoop']
for i,case in enumerate(cases2):
    label,me,fla,_,shift=sgms[case][:5]
    sty=fla2sty[fla]
    i=ys[i]
    sty=('r','s') if i<4 else ('g','o') if i<8 else ('b','d')
    
    mfc='white'
    
    if doShift:
        if len(me)==2:
            me=(me[0]+shift[0],me[1],shift[1])
        elif len(me)==3:
            me=(me[0]+shift[0],me[1],np.sqrt(me[2]**2+shift[1]**2))
    
    if len(me)==2:
        ax.errorbar(x=me[0] ,xerr=me[1],y=[-i], color=sty[0],fmt=sty[1], mfc=mfc)
    elif len(me)==3:
        errTot=np.sqrt(me[1]**2+me[2]**2)
        if doShift:
            print(case,me,errTot)
        ax.errorbar(x=me[0] ,xerr=me[1],y=[-i], color=sty[0],fmt=sty[1], mfc=mfc)
        ax.errorbar(x=me[0] ,xerr=errTot,y=[-i], color=sty[0],fmt=sty[1], mfc=mfc)
    else:
        1/0

ax.yaxis.set_ticks([-i for i in range(len(cases))])
ax.set_yticklabels([sgms[case][0] if case is not None else '' for case in cases],fontsize=12)

# ax.text(72,1-1,r'with $\mathcal{J}_{N\sigma}$',fontsize=15,va='center')
ax.text(72,1-1,r'Direct',fontsize=15,va='center')
for place,text in zip([3,7],['F-H method','Pheno.']):
    ax.fill_between(x=xlim, y1=-place-0.5-1e-5,y2=-place-0.5+1e-5, color='grey')
    ax.text(72,-place-1,text,fontsize=15,va='center')

plt.tight_layout()
plt.savefig(f'{figPath}sgmCompare.pdf' if not doShift else f'{figPath}sgmCompare_shift.pdf')
plt.close()

# B64

In [5]:
Tpack=32; d_jk=1
path='/capstor/store/cscs/userlab/lp139/lyan/code/projectData_old/01_Nsgm/cB211.072.64_base/data_merge/Nsgm_4src_16,18,20.h5'
data=yu.load(path,d=d_jk)
Ncfg,Njk=data['cfgs'][1:]
print(f'Ncfg={Ncfg},Njk={Njk}')

cfg2old=lambda cfg: cfg[1:]+'_r'+{'a':'0','b':'1','c':'2','d':'3'}[cfg[0]]
cfg2new=lambda cfg: {'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]] + cfg[:4]

tfs=[8,10,12,14,16,18,20]
path='/capstor/store/cscs/userlab/lp139/lyan/code/projectData_old/01_Nsgm/cB211.072.64_base/data_from_others/thrp-conn.h5'
with h5py.File(path) as f:
    # cfgs_NJN=[cfg2new(cfg) for cfg in f['gS/up/dt10'].keys()]
    cfgs_old=[cfg2old(cfg) for cfg in data['cfgs'][0]]

    c3ptDic_NJN={tf:np.real(yu.jackknife([f[f'gS/up/dt{tf}/{cfg}'][:] + f[f'gS/dn/dt{tf}/{cfg}'][:] for cfg in cfgs_old],d=d_jk)) for tf in tfs}
    c2ptCorrDic_NJN={tf:np.real(yu.jackknife([f[f'dt{tf}_twop/{cfg}'][()] for cfg in cfgs_old],d=d_jk)) for tf in tfs}

loading: /capstor/store/cscs/userlab/lp139/lyan/code/projectData_old/01_Nsgm/cB211.072.64_base/data_merge/Nsgm_4src_16,18,20.h5
498/498: diags/Z3pt/srcs/Z3pt.h5_NJNpi-Nsrc=4*1                                     
Ncfg=732,Njk=732


In [6]:
flags={
    'cc2pt':True, # average quantities related by complex conjugation for 2pt
    'cc3pt':True, # same for 3pt (Removal of vacuum expectation value requires 'cc2pt'=='cc3pt')
    'll2pt':True, # average irrep rows 'l1' and conjugated 'l2' (Parity breaking effect of tmQCD has been taken care of)
    'll3pt':True, # same for 3pt (This flag has no effect if 'll2pt'=True and spin-projection is done)
    'r2pt': True, # making 2ptMat real for the rest frame # 'll2pt' has to be real for this flag
    'remove_pi0VEV':True, 
    'remove_jVEV':True,
}

def op_remove_pi0(op):
    t=op.split(';')
    if t[-1] in ['p','n','n,pi+']:
        return False
    if t[3] not in ['N0pi0,a','N1pi0,a','N0sgm0,a','N1sgm0,a']:
        return False
    if t[3] in ['N0sgm0,a','N1sgm0,a']:
        t[3]='a'
        t[-1]={'p,sgm':'p'}[t[-1]]
        return ';'.join(t)
    if t[2] == 'G1u':
        t[2]='G1g'
        assert(t[3]=='N0pi0,a'); t[3]='a'
        assert(t[-1]=='p,pi0'); t[-1]='p'
        return ';'.join(t)

def get2pt_diag(opa,opb,diag):
    opab=f'{opa}_{opb}'
    if opab not in data['2pt'].keys() or diag not in data['2pt'][opab].keys():
        return 0
    res=data['2pt'][opab][diag].copy()
    if not flags['remove_pi0VEV']:
        return res
    if diag == 'pi0f-pi0i':
        if opa==opb and opa=='t;0,0,0;pi0':
            res -= (data['VEV']['pi0f']**2)[:,None]
        return res
    
    if 'pi0f' in diag.split('-'):
        t_opa=op_remove_pi0(opa)
        if t_opa != False:
            t=diag.split('-'); t.remove('pi0f'); t_diag='-'.join(t)
            res -= data['2pt'][f'{t_opa}_{opb}'][t_diag] * data['VEV']['pi0f'][opa.split(';')[-1].split(',')[-1]][:,None]
    if 'pi0i' in diag.split('-'):
        t_opb=op_remove_pi0(opb)
        if t_opb != False:
            t=diag.split('-'); t.remove('pi0i'); t_diag='-'.join(t)
            res -= data['2pt'][f'{opa}_{t_opb}'][t_diag] * np.conj(data['VEV']['pi0f'][opb.split(';')[-1].split(',')[-1]][:,None])
    if 'pi0f' in diag.split('-') and 'pi0i' in diag.split('-'):
        t_opa=op_remove_pi0(opa); t_opb=op_remove_pi0(opb)
        if t_opa != False and t_opb != False:
            t=diag.split('-'); t.remove('pi0f'); t.remove('pi0i'); t_diag='-'.join(t)
            res += data['2pt'][f'{t_opa}_{t_opb}'][t_diag] * data['VEV']['pi0f'][opa.split(';')[-1].split(',')[-1]][:,None] * np.conj(data['VEV']['pi0f'][opb.split(';')[-1].split(',')[-1]][:,None])
    return res

def get2pt(opa,opb,diags=yu.diags_all):
    res=np.zeros([Njk,Tpack],dtype=complex)
    opab=f'{opa}_{opb}'; opba=f'{opb}_{opa}'
    if opab in data['2pt']:
        res+=np.sum([get2pt_diag(opa,opb,diag) for diag in data['2pt'][opab].keys() if diag in diags],axis=0)
    if opba in data['2pt']:
        diags_cc={'T', 'T-pi0f'}; 
        res+=np.conj(np.sum([get2pt_diag(opa,opb,diag) for diag in data['2pt'][opba].keys() if diag in diags_cc and diag in diags],axis=0))
    return res

def get2ptMat(ops,diags=yu.diags_all):
    if flags['ll2pt']:
        flags['ll2pt']=False
        ops_flip=[yu.op_flipl(op) for op in ops]
        t=(get2ptMat(ops,diags=diags)+np.conj(get2ptMat(ops_flip,diags=diags)))/2
        flags['ll2pt']=True
        if ops[0].split(';')[1]=='0,0,0' and flags['r2pt']:
            t=np.real(t)
        return t
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    if flags['cc2pt']:
        t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

tfs=[8,10,12,14,16,18,20]

def get3pt_diag(opa,opb,insert,diag):
    if opa in ['g;0,0,0;G1g;a;l1;p','g;0,0,0;G1g;a;l2;p'] and opb in ['g;0,0,0;G1g;a;l1;p','g;0,0,0;G1g;a;l2;p'] and insert.startswith('id_j+_') and diag=='NJN':
        gm,j,tf=insert.split('_')
        res=c3ptDic_NJN[int(tf)]
        return res
    
    opab=f'{opa}_{opb}'
    if opab not in data['3pt'].keys() or diag not in data['3pt'][opab][insert].keys():
        return 0
    res=data['3pt'][opab][insert][diag].copy()
    if flags['remove_pi0VEV']:
        if 'pi0f' in diag.split('-'):
            t_opa=op_remove_pi0(opa)
            if t_opa != False:
                t=diag.split('-'); t.remove('pi0f'); t_diag='-'.join(t)
                res -= data['3pt'][f'{t_opa}_{opb}'][insert][t_diag] * data['VEV']['pi0f'][opa.split(';')[-1].split(',')[-1]][:,None]
        if 'pi0i' in diag.split('-'):
            t_opb=op_remove_pi0(opb)
            if t_opb != False:
                t=diag.split('-'); t.remove('pi0i'); t_diag='-'.join(t)
                res -= data['3pt'][f'{opa}_{t_opb}'][insert][t_diag] * np.conj(data['VEV']['pi0f'][opb.split(';')[-1].split(',')[-1]][:,None])
        if 'pi0f' in diag.split('-') and 'pi0i' in diag.split('-'):
            t_opa=op_remove_pi0(opa); t_opb=op_remove_pi0(opb)
            if t_opa != False and t_opb != False:
                t=diag.split('-'); t.remove('pi0f'); t.remove('pi0i'); t_diag='-'.join(t)
                res += data['3pt'][f'{t_opa}_{t_opb}'][insert][t_diag] * data['VEV']['pi0f'][opa.split(';')[-1].split(',')[-1]][:,None] * np.conj(data['VEV']['pi0f'][opb.split(';')[-1].split(',')[-1]][:,None])
    if flags['remove_jVEV']:
        gm,j,tf=insert.split('_')
        t_insert='_'.join([gm,j])
        if 'j' in diag.split('-') and t_insert in ['id_j+','id_js','id_jc','g5_j-']:
            t=diag.split('-'); t.remove('j'); t_diag='-'.join(t)
            res -= (get2pt_diag(opa,opb,t_diag)[:,int(tf)] * data['VEV']['j'][t_insert])[:,None]
    return res

def get3pt(opa,opb,insert,diags=yu.diags_all):
    res=np.zeros([Njk,int(insert.split('_')[-1])+1],dtype=complex)
    opab=f'{opa}_{opb}'; opba=f'{opb}_{opa}'
    if opab in data['3pt']:
        res+=np.sum([get3pt_diag(opa,opb,insert,diag) for diag in data['3pt'][opab][insert].keys() if diag in diags],axis=0)
    if opba in data['3pt']:
        t=data['3pt'][opb+'_'+opa][insert]; 
        diags_cc={'B3pt','W3pt','Z3pt','T-j', 'T-pi0f-j','T-jPf','B3pt-pi0f','W3pt-pi0f','Z3pt-pi0f'}; 
        if opab not in data['3pt'] or 'NJN-pi0f' not in data['3pt'][opab][insert]:
            diags_cc.add('NJN-pi0i')
        t_add=np.zeros([Njk,int(insert.split('_')[-1])+1],dtype=complex)+np.sum([get3pt_diag(opb,opa,insert,diag) for diag in t.keys() if diag in diags_cc and diag in diags],axis=0)
        res+=np.conj(t_add[:,::-1])*(yu.gtCj[insert.split('_')[0]])
    return res

def get3ptMat(opas,opbs,insert,diags=yu.diags_all):
    if flags['ll3pt']:
        flags['ll3pt']=False
        opas_flip=[yu.op_flipl(op) for op in opas]; opbs_flip=[yu.op_flipl(op) for op in opbs]
        sgns=np.array([[yu.op_getl_sgn(opa)*yu.op_getl_sgn(opb) for opb in opbs] for opa in opas])
        sgns*=yu.fourCPTstar[insert.split('_')[0]]
        t=(get3ptMat(opas,opbs,insert,diags=diags)+np.conj(get3ptMat(opas_flip,opbs_flip,insert,diags=diags))*sgns[None,None,:,:])/2
        flags['ll3pt']=True
        return t
    t=np.transpose([[get3pt(opa,opb,insert,diags) for opb in opbs] for opa in opas],[2,3,0,1])
    if flags['cc3pt']:
        flags['cc3pt']=False
        tt=get3ptMat(opbs,opas,insert,diags)[:,::-1]*(yu.gtCj[insert.split('_')[0]])
        flags['cc3pt']=True
        t=(t+np.conj(np.transpose(tt,[0,1,3,2])))/2
    return t

In [7]:
settings={
    'ylim':[0,60],
}

colorDic={8:'pink',10:'orange',12:'lightgreen',14:'purple',16:'skyblue',18:'brown',20:'blue'}
fmtDic={8:'s',10:'o',12:'d',14:'^',16:'v',18:'<',20:'>'}
# fmtDic={4:'v',5:'d',6:'o',7:'s',8:'<',9:'>',10:'^'}

ops=['g;0,0,0;G1g;a;l1;p','g;0,0,0;G1g;N0sgm0,a;l1;p,sgm']
j='j+'; tfs=[8,10,12,14,16,18,20]

c2ptM=get2ptMat(ops)
# for tf in c2ptCorrDic_NJN.keys():
#     c2ptM[:,tf,0,0]=c2ptCorrDic_NJN[tf]
c3ptM_dic={tf:get3ptMat(ops,ops,f'id_{j}_{tf}',diags=yu.diags_all) for tf in tfs}

GEVP_t0,GEVP_t=7,14
evals,evecs=yu.GEVP(np.mean(c2ptM,axis=0),GEVP_t0)
Revec=evecs[GEVP_t,0,1]/evecs[GEVP_t,0,0]

lbds=[0,0.5,1,1.5]
lbds=[0]
fig, axs = yu.getFigAxs(1,len(lbds))
# yu.addColHeader(axs,lbds)

for i,lbd in enumerate(lbds):
    # print(lbd)
    x=Revec*lbd
    c2pt=np.real(c2ptM[:,:,0,0] + x*c2ptM[:,:,0,1] + x*c2ptM[:,:,1,0] + x**2*c2ptM[:,:,1,1] *0)
    c3ptDic={tf:np.real(c3ptM_dic[tf][:,:,0,0]+ x*c3ptM_dic[tf][:,:,0,1] + x*c3ptM_dic[tf][:,:,1,0]) for tf in tfs}
    c2ptCorrDic={tf:np.real(c2ptCorrDic_NJN[tf] + x*c2ptM[:,tf,0,1] + x*c2ptM[:,tf,1,0]) for tf in tfs}

    ratioCorr={tf:c3ptDic[tf][:,:]/c2ptCorrDic[tf][:,None] for tf in tfs}


    # fig, axs = yu.getFigAxs(1,1)
    # fig.suptitle(f'lambda={lbd}')

    ax=axs[0,i]
    xunit=ens2a[ens]; yunit=[1,0.4638,ens2amu[ens]*ens2aInv[ens],][2]
    ax.set_ylim(settings['ylim'])
    for tf in tfs[:]:
        t_cut=1
        
        ratio=ratioCorr[tf]
        mean,err=yu.jackme(ratio)
        
        if tf%2==0:
            plt_x=np.arange(t_cut - tf//2,tf//2 - (t_cut-1))*xunit; plt_y=mean[t_cut:tf+1-t_cut]*yunit; plt_yerr=err[t_cut:tf+1-t_cut]*yunit
        else:
            plt_x=(np.arange(t_cut - tf//2,tf//2 - (t_cut-1)+1)-0.5)*xunit; plt_y=mean[t_cut:tf+1-t_cut]*yunit; plt_yerr=err[t_cut:tf+1-t_cut]*yunit
        ax.errorbar(plt_x,plt_y,plt_yerr,color=colorDic[tf] if tf in colorDic else None, fmt=fmtDic[tf],mfc='white',label=rf'$t_s/a={tf}$')  

ax.legend(fontsize=12,ncols=len(tfs)//3)
ax.set_xlabel(r'$t_{ins}-t_{s}/2$ [fm]')
ax.set_ylabel(r'$\sigma_{\pi N}$ [MeV]')
plt.savefig(f'{figPath}/sgmB64.pdf',bbox_inches="tight")
plt.close()
None

# BMW_A15

In [8]:
# path='/capstor/store/cscs/userlab/s1174/lyan/code/glwc2/project2/03_NpiScatteringWilson/dataPrepare/A15/data_aux/cfgs_CGWokringCPU'
# with open(path,'r') as f:
#     cfgs=f.read().splitlines()

def symmetrize(dat):
    dat_bw=np.roll(np.flip(dat,axis=1),1,axis=1)
    return((dat[:,:,0]+dat[:,:,5]) - (dat_bw[:,:,10]+dat_bw[:,:,15]))/4

path='/capstor/store/cscs/userlab/lp139/lyan/code/projectData_old/03_NpiScatteringWilson/A15_multiMethod_Nsrc=50/data_merge/NpiScatteringWilson.h5'
data={}
with h5py.File(path) as f:    
    cfgs_all=[cfg.decode() for cfg in f['cfgs'][:]]
    # cfgs_map=[cfgs_all.index(cfg) for cfg in cfgs]
    
    if True:
        cfgs_map=[cfgs_all.index(cfg) for cfg in cfgs_all]
    
    for cont in f['data'].keys():
        if cont in ['N']:
            t=f[f'data/{cont}'][:,:,16]
            t=symmetrize(t)
            data[cont]=yu.jackknife(t[cfgs_map])
            continue
        if cont in ['P','P_hybrid']:
            t=f[f'data/{cont}'][:,:,0]
            t_bw=np.roll(np.flip(t,axis=1),1,axis=1)
            t=(t+t_bw)/2
            data[cont]=yu.jackknife(t[cfgs_map])
            continue
        t=f[f'data/{cont}'][:,:,0]
        t=symmetrize(t)
        data[cont]=yu.jackknife(t[cfgs_map])

In [11]:
contss=[
    ['B114','B122','B132','B214','B232'],
    ['W1141','W1142','W1221','W1222','W1321','W1322','W2321','W2142','W2322'],
    ['Z114','Z122','Z132','Z214','Z222']
]

contss=[
    ['B114'],
]

fig, axs = yu.getFigAxs(1,1, sharex=True)
colors=['r','g','b','purple']
fmts=['s','o','d']
tmin=1; tmax=16
for iconts,conts in enumerate(contss):
    topo=conts[0][0]
    methods={
        'B':['1','2','4'],
        'W':['1','2','3'],
        # 'B':['2','4'],
        # 'W':['2','3'],
        'Z':['1','2']
    }[topo]
    
    for icont, cont in enumerate(conts):
        ax=axs[iconts,icont]
        # ax.title.set_text(f'{cont}')
        for i,method in enumerate(methods):
            dat=np.real(data[f'{cont}_{method}'])
            
            if len(dat.shape)==3:
                # if topo in ['B','W']:
                #     if method == '2':
                #         dat=dat[:,:,:1]
                #     if method in ['3','4']:
                #         dat=dat[:,:,:1]
                #         dat=dat[:,:,:,:1]
                t=dat[:,:,:1]
                t=np.mean(t,axis=2)
            else:
                t=dat[:,:,:1,:1]
                t=np.mean(t,axis=2)
                t=np.mean(t,axis=2)
            mean,err=yu.jackme(t)
            plt_x=np.arange(tmin,tmax)+(i-1)*0.1; plt_y=mean[tmin:tmax]; plt_yerr=err[tmin:tmax]
            ax.errorbar(plt_x,plt_y,plt_yerr,color=colors[i],fmt=fmts[i],label=[r'Stoc$_N$',r'Stoc$_\pi$',r'2Stoc$_\pi$'][i])
                
        ax.legend()
        # ax.set_yscale('log')

ax.set_xlabel(r'$t/a$')
ax.set_ylabel(r'$C_{N\pi,N\pi}$')
plt.savefig(f'{figPath}/B114.pdf',bbox_inches="tight")
plt.close()